# Regresión con scikit-learn

# Dataset: European Soccer Database from Kaggle

**Descripción del Conjunto de Datos:**
- +25.000 partidos
- +10.000 jugadores
- 11 Países
- Temporadas desde 2008 hasta 2016
- Atributos de los jugadores obtenidos del juego FIFA de EA Sports
- Alineación con coordenadas (X, Y)
- Estadísticas detalladas de cada partido


**Fuente:** <a href="https://www.kaggle.com">Kaggle</a>

**Más información:**
<a href="https://www.kaggle.com/hugomathien/soccer"> European Soccer Database</a>

# Importar librerías

In [ ]:
%matplotlib inline
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import minmax_scale
from math import sqrt

# Importar datos

In [ ]:
# Crear una Conexión a la Base de Datos
cnx = sqlite3.connect('../datasets/soccer/database.sqlite')
# Importar los Datos de la Base de Datos
df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

# Definir las variables que usaremos

In [ ]:
features = [
       'potential', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes']

# Definir la variable objetivo

In [ ]:
target = ['overall_rating']

# Limpiar los datos

In [ ]:
df = df.dropna()

# Extraer los valores de las variables

In [ ]:
X = df[features]

# Extraer los valores del objetivo

In [ ]:
y = df[target]

# Consultar una muestra de las variables

In [ ]:
X.iloc[2]

# Consultar algunos valores del objetivo

In [ ]:
y.head()

In [ ]:
y.tail()

# Dividir el conjunto de datos en Entrenamiento y Evaluación

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=324)

# Regresión Lineal
## La regresión líneal se usa para ajustar una línea a los datos
<img src="../images/linear-regression.png" align="middle" style="width:550px;height:360px"/>

## Crear el modelo con los datos de entrenamiento

In [ ]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

## Ejecutar el modelo con los datos de evaluación

In [ ]:
y_prediction = regressor.predict(X_test)
y_prediction

## Calcular estadísticas básicas de los datos de test

In [ ]:
y_test.describe()

## Evaluar la precisión del módelo

In [ ]:
RMSE = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))

In [ ]:
print(RMSE)

# Decision Tree Regression
## El arbol de decisión se usa para ajustar una curva a los datos
<img src="../images/decision-tree-regression.png" align="middle" style="width:550px;height:360px"/>

## Crear el modelo con los datos de entrenamiento

In [ ]:
regressor = DecisionTreeRegressor(max_depth=20)
regressor.fit(X_train, y_train)

## Ejecutar el modelo con los datos de evaluación

In [ ]:
y_prediction = regressor.predict(X_test)
y_prediction

## Calcular estadísticas básicas de los datos de test

In [ ]:
y_test.describe()

## Evaluar la precisión del modelo

In [ ]:
error = mean_squared_error(y_true = y_test, y_pred = y_prediction)

In [ ]:
print(error)